In [1]:
import pandas as pd
data=pd.read_csv("FAOLEX_Agriculture.csv")
print(data.keys())

Index(['Record Id', 'Record URL', 'Document URL', 'Title', 'Original title',
       'Date of original text', 'Last amended date', 'Available website',
       'Language of document', 'Country/Territory', 'Regional organizations',
       'Territorial subdivision', 'Type of text', 'Repealed', 'Abstract',
       'Primary subjects', 'Domain', 'Keywords'],
      dtype='object')


In [164]:
import nltk
keywords=list(data["Keywords"])
country=list(data["Country/Territory"])
i=0
# Split alle keywords op i en liste og ændrer nan til at være en str med Nan
for words in keywords:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    if type(country[i])!=str:
        country[i]="NaN"
    keywords[i]=words.split(";")
    i+=1



In [165]:
i=0

for words in keywords:
    keywords[i]=words + [" "+country[i]]
    i+=1

In [166]:
print(keywords[13])

['agricultural commodities', ' international trade', ' certification', ' international trade', ' certification', ' international trade', ' certification', ' international trade', ' packaging/labelling', ' certification', ' international trade', ' certification', ' international trade', ' packaging/labelling', ' certification', ' pests/diseases', ' plant protection', ' Thailand']


In [33]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import numpy as np
stemmer = SnowballStemmer("english")
stemmed_words=[]
for i in range(len(keywords)):
    stemmed_words.append([stemmer.stem(word) for word in keywords[i]])



In [151]:
print(keywords[14083])
print(stemmed_words[14083])

['forest management/forest conservation', ' forestry protection measures', ' protection forest', ' institution', ' inspection', ' enforcement/compliance', 'China']
['forest management/forest conserv', ' forestry protection measur', ' protection forest', ' institut', ' inspect', ' enforcement/compli']


In [208]:
# Find number of extinct domains
a= list(set(data["Repealed"]))
i=0
for words in a:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    a[i]=words.split(";")

    i+=1



In [209]:
# print different domains
a=sum(a,[])
i=0
for word in a:
        word=word.strip()
        a[i]=word
        i+=1

print(set(a))




{'Nan', 'Y'}


In [37]:
# Count number documents in a language
languages=list(data["Language of document"])
print(languages.count("English"))

26273


In [167]:
# Make a top2vec topic model
from top2vec import Top2Vec
i=0
for word in keywords:
    keywords[i]="".join(word)
    i+=1
model = Top2Vec(keywords,embedding_model="doc2vec")


2023-03-23 09:47:48,085 - top2vec - INFO - Pre-processing documents for training
2023-03-23 09:47:52,033 - top2vec - INFO - Creating joint document/word embedding
2023-03-23 09:49:31,144 - top2vec - INFO - Creating lower dimension embedding of documents
2023-03-23 09:50:26,516 - top2vec - INFO - Finding dense areas of documents
2023-03-23 09:50:30,979 - top2vec - INFO - Finding topics


In [182]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_sizes

array([766, 705, 660, 627, 597, 566, 512, 480, 477, 461, 451, 433, 418,
       415, 397, 357, 354, 348, 347, 340, 339, 334, 331, 331, 330, 327,
       322, 320, 314, 312, 307, 302, 301, 295, 295, 292, 288, 288, 287,
       284, 278, 278, 277, 276, 276, 275, 270, 267, 265, 259, 259, 257,
       257, 255, 255, 254, 253, 249, 247, 243, 242, 240, 240, 239, 238,
       238, 238, 237, 237, 236, 231, 230, 228, 224, 223, 223, 223, 222,
       221, 218, 218, 218, 216, 216, 216, 215, 214, 212, 212, 211, 209,
       208, 207, 207, 206, 205, 205, 204, 202, 202, 201, 200, 200, 199,
       199, 198, 198, 196, 195, 192, 192, 191, 191, 191, 191, 190, 189,
       189, 188, 188, 187, 187, 187, 187, 186, 185, 183, 183, 182, 181,
       180, 180, 178, 178, 178, 178, 178, 177, 176, 176, 176, 176, 175,
       175, 174, 173, 173, 173, 172, 171, 170, 170, 169, 168, 166, 166,
       166, 165, 164, 164, 163, 162, 162, 161, 161, 160, 160, 160, 159,
       158, 158, 157, 157, 156, 156, 155, 155, 154, 154, 154, 15

In [183]:
topic_words,word_scores,topic_nums = model.get_topics(10)

In [184]:
print(topic_words)

[['great' 'kingdom' 'ireland' 'britain' 'northern' 'jersey' 'united'
  'swine' 'slaughtering' 'equine' 'animal' 'disinfection' 'feedstuffs'
  'steroids' 'hormones' 'goats' 'feed' 'sheep' 'disinfestation' 'meat'
  'cattle' 'limit' 'vaccination' 'camelids' 'drugs' 'inspection'
  'welfare' 'guernsey' 'poultry' 'reproduction' 'veterinary' 'herd'
  'svalbard' 'additives' 'rodents' 'traceability' 'mrl' 'maximum' 'pests'
  'diseases' 'quarantine' 'health' 'residue' 'book' 'edible' 'birds'
  'treated' 'residues' 'procedures' 'is']
 ['port' 'transshipment' 'unreported' 'illegal' 'unregulated' 'iuu'
  'foreign' 'landing' 'fishing' 'high' 'state' 'seas' 'bycatch'
  'migratory' 'marine' 'cartilaginous' 'nan' 'vessel' 'fisheries'
  'fishery' 'harbour' 'svalbard' 'catch' 'territorial' 'exclusive'
  'fishes' 'allowable' 'eez' 'economic' 'marshall' 'artisanal' 'palau'
  'seychelles' 'size' 'method' 'mammals' 'total' 'gear' 'enforcement'
  'charge' 'sea' 'vanuatu' 'maritime' 'kiribati' 'zone' 'federate

In [186]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=2, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 75345, Score: 0.949529230594635
-----------
fishery management and conservation marine fisheries territorial sea EEZ-Exclusive Economic Zone maritime zone sovereignty Peru
-----------

Document: 74771, Score: 0.9338657259941101
-----------
marine fisheries EEZ-Exclusive Economic Zone maritime zone territorial sea continental shelf data collection/reporting EEZ-Exclusive Economic Zone maritime zone territorial sea continental shelf Russian Federation
-----------

Document: 43109, Score: 0.932529866695404
-----------
authorization/permit territorial sea EEZ-Exclusive Economic Zone fishery management and conservation fishing area Norway
-----------

Document: 36836, Score: 0.9314826726913452
-----------
EEZ-Exclusive Economic Zone territorial sea continental shelf marine fisheries marine fishes research fishing vessel allocation/quota fishing gear/fishing method fishing authorization fishery management and conservation Russian Federation
-----------

Document: 74773, Score: 0.92

In [195]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["denmark"], num_topics=3)
print(topic_words)
print(topic_nums)

[array(['greenland', 'allocation', 'catch', 'fishing', 'allowable',
       'fishery', 'bycatch', 'quota', 'fishes', 'crustaceans', 'marine',
       'total', 'size', 'cartilaginous', 'mesh', 'fisheries', 'svalbard',
       'seasons', 'gear', 'diadromous', 'method', 'vessel', 'landing',
       'molluscs', 'prohibited', 'commercial', 'denmark', 'turtles',
       'transshipment', 'charge', 'migratory', 'iceland', 'mammals',
       'area', 'foreign', 'norway', 'territorial', 'port', 'conservation',
       'seas', 'sea', 'exclusive', 'high', 'fee', 'eez', 'harbour', 'is',
       'jurisdiction', 'continental', 'shelf'], dtype='<U15'), array(['qatar', 'bahrain', 'kuwait', 'nuclear', 'oman', 'sweden',
       'arabia', 'singapore', 'procedural', 'emirates', 'matters',
       'natural', 'iraq', 'greece', 'saudi', 'energy', 'gas', 'denmark',
       'coral', 'hydropower', 'latvia', 'radiation', 'coal', 'palestine',
       'aquatic', 'israel', 'prohibited', 'fee', 'north', 'sinking',
       'guam', 

In [196]:
topic_mapping = model.hierarchical_topic_reduction(num_topics=20)

In [197]:
print(model.topic_words_reduced)

[['steroids' 'hormones' 'mrl' 'residue' 'drugs' 'limit' 'hygiene'
  'maximum' 'swine' 'procedures' 'residues' 'sanitary' 'feed'
  'feedstuffs' 'veterinary' 'animal' 'toxicity' 'poultry' 'poisoning'
  'equine' 'slaughtering' 'meat' 'cattle' 'sheep' 'vaccination' 'goats'
  'additives' 'health' 'dairy' 'mercosur' 'safety' 'disinfestation'
  'codex' 'inspection' 'rodents' 'treated' 'haccp' 'disinfection'
  'andean' 'edible' 'packaging' 'pests' 'traceability' 'trade'
  'antimicrobial' 'diseases' 'welfare' 'labelling' 'milk' 'alimentarius']
 ['equine' 'sheep' 'swine' 'goats' 'cattle' 'rodents' 'herd' 'camelids'
  'book' 'reproduction' 'poultry' 'animal' 'slaughtering' 'vaccination'
  'mercosur' 'animals' 'welfare' 'meat' 'health' 'disinfestation' 'birds'
  'jersey' 'hygiene' 'sericulture' 'apiculture' 'steroids' 'hormones'
  'drugs' 'procedures' 'disinfection' 'quarantine' 'identification'
  'north' 'pests' 'feed' 'sanitary' 'ranching' 'inspection' 'marking'
  'captive' 'diseases' 'edible' '

In [202]:
model.topic_words_reduced[4]

array(['viticulture', 'oenological', 'tribunal', 'court', 'herzegovina',
       'beverages', 'tobacco', 'geographical', 'bosnia', 'indication',
       'fraud', 'settlement', 'codex', 'inspection', 'lao',
       'compensation', 'dispute', 'intellectual', 'labelling', 'trade',
       'certification', 'justice', 'practices', 'consumer', 'lanka',
       'liability', 'packaging', 'deceit', 'information', 'plant',
       'haccp', 'traceability', 'alimentarius', 'offences', 'internal',
       'tristan', 'adulteration', 'classification', 'singapore',
       'breeders', 'access', 'seeds', 'commodities', 'vested', 'sugar',
       'floriculture', 'veterinary', 'liechtenstein', 'legal',
       'agricultural'], dtype='<U14')

In [82]:
topic_words,word_scores,topic_nums = model.get_topics(10)
topic_words

array([['mesh', 'fishing', 'seasons', 'size', 'fishery', 'method',
        'cartilaginous', 'catch', 'bycatch', 'fisheries', 'landing',
        'prohibited', 'marine', 'total', 'allowable', 'charge',
        'allocation', 'fishes', 'vessel', 'gear', 'quota', 'artisanal',
        'foreign', 'commercial', 'diadromous', 'port', 'migratory',
        'transshipment', 'turtles', 'mammals', 'molluscs', 'crustaceans',
        'illegal', 'high', 'seas', 'inland', 'iuu', 'territorial',
        'unregulated', 'area', 'exclusive', 'state', 'eez',
        'enhancement', 'management', 'conservation', 'coral',
        'unreported', 'repopulation', 'sea'],
       ['floriculture', 'planting', 'seeds', 'material', 'fibre',
        'plant', 'cane', 'beet', 'variety', 'vegetables', 'breeders',
        'oleaginous', 'legumes', 'fruits', 'edible', 'nuts', 'sugar',
        'certification', 'rice', 'tobacco', 'cereals', 'labelling',
        'weeds', 'crops', 'oenological', 'production', 'packaging',
        '

In [274]:
# Tæl hvor mange der er repelaed inden for et dato interval
dates=data["Date of original text"]
repealed=data["Repealed"]
antal=0
for i in range(1800,1970):
    c=dates==str(i)
    d=repealed=="Y"
    antal+=sum(c*d)
print(antal)


91
